# Recommender Systems - notebook 1 - Traditional Approaches

In [24]:
"""
(Practical tip) Table of contents can be compiled directly in jupyter notebooks using the following code:
I set an exception: if the package is in your installation you can import it otherwise you download it 
then import it.
"""
try:
    from jyquickhelper import add_notebook_menu 
except:
    !pip install jyquickhelper
    from jyquickhelper import add_notebook_menu
    
"""
Output Table of contents to navigate easily in the notebook. 
For interested readers, the package also includes Ipython magic commands to go back to this cell
wherever you are in the notebook to look for cells faster
"""
add_notebook_menu()

## Imports

In [25]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [26]:
from tqdm import tqdm
import pandas as pd
import numpy as np



## Dataset description

We use here the MovieLens 100K Dataset. It contain 100,000 ratings from 1000 users on 1700 movies.

* u.train / u.test part of the original u.data information
    * The full u data set, 100000 ratings by 943 users on 1682 items.
              Each user has rated at least 20 movies.  Users and items are
              numbered consecutively from 1.  The data is randomly
              ordered. This is a tab separated list of 
	         user id | item id | rating | timestamp. 
              The time stamps are unix seconds since 1/1/1970 UTC   
* u.info
    * The number of users, items, and ratings in the u data set.
* u.item
    * Information about the items (movies); this is a tab separated
              list of
              movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
              The last 19 fields are the genres, a 1 indicates the movie
              is of that genre, a 0 indicates it is not; movies can be in
              several genres at once.
              The movie ids are the ones used in the u.data data set.
* u.genre
    * A list of the genres.
* u.user
    * Demographic information about the users; this is a tab
              separated list of
              user id | age | gender | occupation | zip code
              The user ids are the ones used in the u.data data set.

In [27]:
path = "https://www.i3s.unice.fr/~riveill/dataset/dataset_movilens_100K/" 

Before we build our model, it is important to understand the distinction between implicit and explicit feedback, and why modern recommender systems are built on implicit feedback.

* **Explicit Feedback:** in the context of recommender systems, explicit feedback are direct and quantitative data collected from users.
* **Implicit Feedback:** on the other hand, implicit feedback are collected indirectly from user interactions, and they act as a proxy for user preference. 

The advantage of implicit feedback is that it is abundant. Recommender systems built using implicit feedback allow recommendations to be adapted in real time, with each click and interaction.

Today, online recommender systems are built using implicit feedback.

### Data preprocessing

In [28]:
# Load data
np.random.seed(123)

ratings = pd.read_csv(path+'u.data', sep='\t', header=None,
                            names=['userId', 'movieId', 'rating', 'timestamp'])
ratings = ratings.sort_values(['timestamp'], ascending=True)
print("Nb ratings:", len(ratings))
ratings.head()

Nb ratings: 100000


,userId,movieId,rating,timestamp
214,259,255,4,874724710
83965,259,286,4,874724727
43027,259,298,4,874724754
21396,259,185,4,874724781
82655,259,173,4,874724843


### Data splitting

Separating the dataset between train and test in a random fashion would not be fair, as we could potentially use a user's recent evaluations for training and previous evaluations. This introduces a data leakage with an anticipation bias, and the performance of the trained model would not be generalizable to real world performance.

Therefore, we need to slice the train and test based on the timestamp


In [29]:
# Split dataset
train_ratings, test_ratings = np.split(ratings, [int(.9*len(ratings))])

max(train_ratings['timestamp']) <= min(test_ratings['timestamp'])

True

In [30]:
# drop columns that we no longer need
train_ratings = train_ratings[['userId', 'movieId', 'rating']]
test_ratings = test_ratings[['userId', 'movieId', 'rating']]

len(train_ratings), len(test_ratings)

(90000, 10000)

In [31]:
# Get a list of all movie IDs
all_movieIds = ratings['movieId'].unique()

### Build pivot table

In [32]:
''' Pivot table for train set '''
train_pivot = pd.pivot_table(train_ratings, values='rating', index='userId', columns='movieId')
print("Nb users: ", train_pivot.shape[0])
print("Nb movies:", train_pivot.shape[1])
display(train_pivot.head())

Nb users:  867
Nb movies: 1637


movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1652,1653,1655,1656,1657,1658,1659,1660,1661,1662,1664,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4.0,NaN,NaN,NaN,NaN,NaN,2.0,4.0,4.0,NaN,NaN,4.0,2.0,5.0,3.0,NaN,NaN,NaN,4.0,NaN,3.0,3.0,4.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
train_users = train_pivot.index
train_movies = train_pivot.columns
print(train_users)
print(train_movies)

Int64Index([  1,   2,   3,   5,   6,   7,   8,   9,  10,  12,
            ...
            934, 935, 936, 937, 938, 939, 940, 941, 942, 943],
           dtype='int64', name='userId', length=867)
Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682],
           dtype='int64', name='movieId', length=1637)


In [34]:
#pd.pivot.mean() -> columnlara uygulanir

## Collaborative filtering based on Users similarity

This approach uses scores that have been assigned by other users to calculate predictions.

In pivot table
* Rows are users, $u, v$
* Columns are items, $i, j$

$$pred(u, i) = \frac{\sum_v sim(u,v)*r_{v,i}}{\sum_v sim(u,v)}$$

Wich similarity function:

* Euclidean distance $[0,1]$: $sim(a,b)=\frac{1}{1+\sqrt{\sum_i (r_{a,i}-r_{b,i})^2}}$
* Pearson correlation $[-1,1]$: $sim(a,b)=\frac{\sum_i (r_{a,i}-r_a)(r_{b,i}-r_b)}{{\sum_i (r_{a,i}-r_a)^2}{\sum_i (r_{b,i}-r_b)^2}}$
* Cosine similarity $[-1,1]$: $sim(a, b)=\frac{a.b}{|a|.|b|}$

Which function should we use? The answer is that there is no fixed recipe; but there are some issues we can take into account when choosing the proper similarity function. On the one hand:
* Pearson correlation usually works better than Euclidean distance since it is based more on the ranking than on the values. So, two users who usually like more the same set of items, although their rating is on different scales, will come out as similar users with Pearson correlation but not with Euclidean distance.
* On the other hand, when dealing with binary/unary data, i.e., like versus not like or buy versus not buy, instead of scalar or real data like ratings, cosine distance is usually used.

### Build predictor

In [35]:
# Step 1: build the similarity matrix between users

#train_pivot.transpose() : this makes userId as features so as we calculte corr matrix among features.

correlation_matrix = train_pivot.transpose().corr('pearson')
correlation_matrix.head()

userId,1,2,3,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28,29,30,31,32,34,35,36,37,40,41,42,43,44,45,46,...,900,901,902,903,904,905,906,907,908,910,912,913,914,915,916,917,919,920,921,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.160841,0.112780,0.420809,0.287159,1.237128e-01,0.692086,-0.102062,-0.092344,4.219129e-02,0.429857,0.041761,0.145805,0.142875,0.578641,-0.036810,-0.577350,0.035265,0.533282,0.591613,0.289164,-0.073681,0.166581,0.095967,0.317511,6.559565e-01,0.544982,-0.092214,0.632529,0.891042,-0.576461,-0.944911,0.156751,0.427312,0.061102,0.279820,0.150242,0.373536,0.495303,0.337941,...,0.291404,0.181501,-0.098326,0.376552,-0.586078,0.628773,0.036124,0.224023,0.289583,0.401217,-0.249261,0.209879,-0.606339,NaN,0.415419,0.079444,0.174609,0.750000,0.180056,0.590413,0.270584,0.297775,0.645179,0.137312,-1.260631e-01,0.296122,0.085688,-0.116027,0.307356,0.392989,0.061695,-0.260242,0.383733,2.899974e-02,0.326744,5.343904e-01,0.263289,0.205616,-0.180784,0.067549
2,0.160841,1.000000,0.067420,0.327327,0.446269,4.807341e-01,0.585491,0.242536,0.668145,3.532708e-16,0.364341,0.025832,0.196767,0.249068,-0.174928,0.489976,-0.241747,0.207514,0.214923,0.801784,0.681097,0.813220,-0.441942,0.072548,0.321960,-1.998558e-16,0.176958,0.577350,0.220564,0.812037,0.000000,0.102062,0.426401,-0.327510,0.611111,-0.117851,-0.055297,0.244677,0.778625,0.659082,...,0.000000,0.137460,0.490910,0.408815,-0.065795,0.610572,0.137018,0.163163,0.316228,0.807031,NaN,-0.079181,1.000000,0.196748,0.613711,0.217195,0.500326,0.225374,-0.170093,0.383184,0.484162,-1.000000,0.192487,0.063296,8.164966e-01,0.904534,0.069224,0.200900,-0.218218,0.900539,0.021007,-0.271163,0.214017,5.616449e-01,0.331587,-7.671236e-18,-0.011682,-0.062017,0.085960,0.479702
3,0.112780,0.067420,1.000000,NaN,-0.109109,-5.037555e-17,0.291937,NaN,0.311086,1.889822e-01,-0.045419,0.440225,0.275269,-0.280579,0.188982,-0.301511,-0.774597,0.500000,-0.096962,0.500000,0.365148,0.547723,NaN,-0.406202,0.108465,2.657887e-01,-0.395285,-0.525885,0.438529,0.199310,-0.224807,0.149071,NaN,-0.300123,NaN,1.000000,-0.226461,0.283473,1.000000,0.306719,...,0.750568,-0.547723,0.161861,-0.214423,-0.986440,0.156941,0.944911,-0.579860,0.262613,-0.386510,1.000000,-0.229972,NaN,0.032291,0.000000,-1.000000,-0.046809,0.193246,0.790184,0.568038,0.557086,-0.447914,0.201498,-1.000000,5.233642e-17,1.000000,-0.188982,-0.187767,NaN,-0.500000,NaN,NaN,-0.045162,-5.233642e-17,-0.137523,NaN,-0.104678,1.000000,-0.011792,NaN
5,0.420809,0.327327,NaN,1.000000,0.241817,1.490373e-01,0.537400,0.577350,0.087343,1.169812e-01,0.405006,0.399659,0.392746,-0.145066,0.834441,0.241597,0.071429,0.265519,0.534489,0.461165,0.281932,-0.175161,0.334947,0.632746,0.523056,NaN,0.190285,0.866025,0.353704,NaN,NaN,NaN,0.266471,NaN,0.165663,0.131793,0.281339,0.476972,0.599693,NaN,...,0.469809,0.099976,-0.710148,0.210058,-0.724090,NaN,0.870388,-0.430077,0.100175,-0.015778,-0.540467,-0.101824,-0.158114,NaN,0.151851,0.000000,0.324017,NaN,0.270785,0.569898,0.221123,0.875000,NaN,0.166794,2.000000e-01,-0.064550,0.264594,0.913500,0.347306,0.293257,0.229532,-0.500000,0.439286,6.085806e-01,0.484211,8.807048e-01,0.027038,0.468521,0.318163,0.346234
6,0.287159,0.446269,-0.109109,0.241817,1.000000,1.758687e-01,0.687745,0.132353,0.273987,1.121937e-01,0.272046,0.012967,0.157704,0.303696,0.297719,0.192872,0.397149,0.083527,0.530130,0.572611,0.450571,0.257463,-0.142939,0.349448,0.297069,1.611432e-01,0.366139,0.562300,0.235198,0.577897,-0.628971,-0.316228,0.589378,0.060994,0.391744,0.175191,0.045716,0.131194,0.460409,0.436292,...,0.207888,0.147796,0.410710,0.355657,0.026780,0.078811,-0.007265,0.010430,0.247910,0.248401,0.171527,0.206266,0.944911,0.389249,0.464018,-0.162507,0.062726,0.131940,0.064843,0.147576,0.402318,1.000000,0.551825,-0.249126,-3.510989e-01,0.283363,0.084928,0.365921,0.166989,0.301821,-0.064033,-0.264292,0.3

In [36]:
# Step2 build rating function
# We want to calculate the rating that a user could have made for an item.

# Il est plus efficace de travailler avec numpy qu'avec pandas.
# On transforme donc la matrice pivot en numpy
pivot = train_pivot.to_numpy()
# idem pour la matrice de correlation
corr = correlation_matrix.to_numpy()
# Malheureusement, on doit utiliser 2 dictionnaires pour passer
# Du nom de la colonne movieId dans son indice en numpy
movie2column = {j:i for i, j in enumerate(train_pivot.columns)}
# Du nom de la ligne userId dans son indice en numpy
user2row = {j:i for i, j in enumerate(train_pivot.index)}


def predict(pivot, corr, userId, movieId):
    if movieId in movie2column.keys():
        movie = movie2column[movieId]
    else:
        return 2.5
    if userId in user2row.keys():
        user = user2row[userId]
    else:
        return 2.5

    # Normalement le rating est inconnu
    if np.isnan(pivot[user, movie]):
        num = 0
        den = 0
        for u in range(len(corr)):
            if not np.isnan(pivot[u,movie]) and not np.isnan(corr[user,u]):
                # Si l'utilisateur u a déjà vu le film movie
                # et si les deux utilisateurs ont au moins vu un même film
                
                num += corr[user,u] * pivot[u,movie]
                den += abs(corr[user,u])
                
        if den != 0:
            return num/den  
        else:
            return 2.5 # default value
    else:
        # le film a déjà été vu
        print("l'utilisateur", userId, "a déjà vu le film", movieId, "et lui a donné la note de", pivot[user, movie])
        return pivot[user, movie]
        
predict(pivot, corr, 1, 1)
predict(pivot, corr, 3, 28)

l'utilisateur 1 a déjà vu le film 1 et lui a donné la note de 5.0


1.8527972301545377

### Predict

In [37]:
# Step 3 add the predicted rating to the test set
pred = []
for _, userId, movieId, _ in tqdm(test_ratings[['userId', 'movieId', 'rating']].itertuples()):
    pred += [predict(pivot, corr, userId, movieId)]

test_ratings['User based'] = pred
test_ratings.head()

10000it [00:06, 1594.00it/s]


,userId,movieId,rating,User based
557,90,900,4,0.657995
6675,90,269,5,0.987827
562,90,289,3,0.572064
62660,90,270,4,1.077609
68756,90,268,4,1.202310


### Evaluate the predictor

Now that we have trained our model, assigned a value to each pair (userId, movieId), we are ready to evaluate it.

#### Evaluation with classical metrics: RMSE

In traditional machine learning projects, we evaluate our models using measures such as accuracy (for classification problems) and RMSE (for regression problems). This is what we will do in the first instance.

In [38]:
# Step 4 evaluate the resulte : with classical metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

print("RMSE:",
      np.sqrt(mean_squared_error(test_ratings['rating'], test_ratings['User based'])),
     )

RMSE: 1.7798966446725806


#### Hit Ratio @ K

However, a measure such as RMSE does not provide a satisfactory evaluation of recommender systems. To design a good metric for evaluating recommender systems, we need to first understand how modern recommender systems are used.

Amazon, Netflix and others uses a list of recommendations. The key here is that we don’t need the user to interact with every single item in the list of recommendations. Instead, we just need the user to interact with at least one item on the list — as long as the user does that, the recommendations have worked.

To simulate this, let’s run the following evaluation protocol to generate a list of top 10 recommended items for each user.

* For each user, randomly select 99 items that the user has not interacted with.
* Combine these 99 items with the test item (the actual item that the user last interacted with). We now have 100 items.
* Run the model on these 100 items, and rank them according to their predicted probabilities.
* Select the top 10 items from the list of 100 items. If the test item is present within the top 10 items, then we say that this is a hit.
* Repeat the process for all users. The Hit Ratio is then the average hits.

This evaluation protocol is known as **Hit Ratio @ K**, and it is commonly used to evaluate recommender systems.

<font color='red'>
$TO DO - Students$

* Fill the gaps
</font>

In [39]:
# Step 2 with hit ratio
def HRatio(test_ratings, predict, K=10):
    # User-item pairs for testing
    test_user_item_set = set(list(set(zip(test_ratings['userId'], test_ratings['movieId'])))[:1000])
    
    # Dict of all items that are interacted with by each user
    user_interacted_items = ratings.groupby('userId')['movieId'].apply(list).to_dict()
    hits = []
    for (u,i) in tqdm(test_user_item_set):
        interacted_items = user_interacted_items[u]
        not_interacted_items = set(all_movieIds) - set(interacted_items)
        selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
        test_items = selected_not_interacted + [i]
        predicted_labels = predict([np.array([u]*100), np.array(test_items)]).reshape(-1)
        topK_items = [test_items[i] for i in np.argsort(predicted_labels)[-K:].tolist()]
        
        if i in topK_items:
            hits.append(1)
        else:
            hits.append(0)
    hr = np.average(hits)
    print("The Hit Ratio @ {} is {:.2f}".format(K, hr))
    return hr

In [40]:
def predictor(pairs):
    pred = []
    for userId, movieId in zip(pairs[0], pairs[1]):
        pred += [predict(pivot, corr, userId, movieId)]
    return np.array(pred)

HR = dict()
hr = HRatio(test_ratings, predictor, 25)
HR['User based'] = hr

100%|██████████| 1000/1000 [01:07<00:00, 14.87it/s]

The Hit Ratio @ 25 is 0.78


## Improve the rating

In [41]:
test_ratings.head()

,userId,movieId,rating,User based
557,90,900,4,0.657995
6675,90,269,5,0.987827
562,90,289,3,0.572064
62660,90,270,4,1.077609
68756,90,268,4,1.202310


### Trick 1:

Since humans do not usually act the same as critics, i.e., some people usually rank movies higher or lower than others, this prediction function can be easily improved by taking into account the user mean as follows:

$$pred(u, i) = \overline{r_u} + \frac{\sum_v sim(u,v)*(r_{v,i} - \overline{r_v})}{\sum_v sim(u,v)}$$

<font color='red'>
$TO DO - Students$

* Modify the previous code in order to implement "Trick 1"
</font>

In [42]:
""" FILL """

def predict_trick1(pivot, corr, userId, movieId):
    if movieId in movie2column.keys():
        movie = movie2column[movieId]
    else:
        return 2.5
    if userId in user2row.keys():
        user = user2row[userId]
    else:
        return 2.5

    # Normalement le rating est inconnu
    if np.isnan(pivot[user, movie]):
        num = 0
        den = 0
        for u in range(len(corr)):
            if not np.isnan(pivot[u,movie]) and not np.isnan(corr[user,u]):
                # Si l'utilisateur u a déjà vu le film movie
                # et si les deux utilisateurs ont au moins vu un même film
                #print(pd.pivot.mean())
                
                num += corr[user,u] * ( pivot[u,movie] - np.nanmean(pivot[u:]) )
                den += abs(corr[user,u])     
        if den != 0:
          #r_u
            return (num/den ) + np.nanmean(pivot[user:])
        else:
            return 2.5 # default value
    else:
        # le film a déjà été vu
        print("l'utilisateur", userId, "a déjà vu le film", movieId, "et lui a donné la note de", pivot[user, movie])
        return pivot[user, movie]
        
predict_trick1(pivot, corr, 1, 1)
predict_trick1(pivot, corr, 3, 28)

l'utilisateur 1 a déjà vu le film 1 et lui a donné la note de 5.0


3.7032962034907273

### Trick 2:

If two users have very few items in common, let us imagine that there is only one, and the rating is the same, the user similarity will be really high; however, the confidence is really small. It's possible to add a ponderation coefficient.

$$newsim(a, b) = sim(a,b) * \frac{min(N, |P_{a,b}|)}{N}$$ 

where $|P_{a,b}|$ is the number of common items shared by user a and user b. The coefficient is $< 1$ if the number of common movies is $< N$ and $1$ otherwise.

In [43]:
# Count the number of common items shared by user 1 and user 2.
pivot = train_pivot.to_numpy()
movie2column = {j:i for i, j in enumerate(train_pivot.columns)}
user2row = {j:i for i, j in enumerate(train_pivot.index)}
a = user2row[1]
b = user2row[2]
common = pivot[a,:] + pivot[b,:]
common = np.count_nonzero(~np.isnan(common))
common

18

<font color='red'>
$TO DO - Students$

* Modify the previous code in order to implement "Trick 2"
</font>

In [44]:
#This fucntion finds the number of common items between user a and user b.
#|P_ab|
def num_common_items(a, b):
  user2row = {j:i for i, j in enumerate(train_pivot.index)}
  a1 = user2row[a]
  b1 = user2row[b]
  common = pivot[a1,:] + pivot[b1,:]
  common = np.count_nonzero(~np.isnan(common))
  return(common)

In [46]:
def predict_trick2(pivot, corr, userId, movieId):
    if movieId in movie2column.keys():
        movie = movie2column[movieId]
    else:
        return 2.5
    if userId in user2row.keys():
        user = user2row[userId]
    else:
        return 2.5

    # Normalement le rating est inconnu
    if np.isnan(pivot[user, movie]):
        num = 0
        den = 0
        for u in range(len(corr)):
            #for 
            if not np.isnan(pivot[u,movie]) and not np.isnan(corr[user,u]):
                # Si l'utilisateur u a déjà vu le film movie
                # et si les deux utilisateurs ont au moins vu un même film
                #print(pd.pivot.mean())
                N=10.0
                new_sim = corr[user,u] * min(N, num_common_items(user,u)) / N
                num += new_sim *  pivot[u,movie]  
                den += abs(new_sim)
                print(new_sim)
                
        if den != 0:
          #r_u
            return (num/den ) 
        else:
            return 2.5 # default value
    else:
        # le film a déjà été vu
        print("l'utilisateur", userId, "a déjà vu le film", movieId, "et lui a donné la note de", pivot[user, movie])
        return pivot[user, movie]
        
predict_trick2(pivot, corr, 1, 1)
predict_trick2(pivot, corr, 3, 28)

l'utilisateur 1 a déjà vu le film 1 et lui a donné la note de 5.0


KeyError: ignored

## To go further

1. Do the same, but with correlation between items. It's Collaborative filtering based on Items similarity. It's also possible to use the 2 previous trick

1.  Use Matrix factorization: decompose R in P, Q at rank k (i.e. if R is a m.n matrix, P is a m.k matrix and Q is a n.k matrix) the reconstruct R with P and Q (i.e. $\hat{R} = P Q^T$)

1. Use Matrix decomposition: do an truncated SVD decomposition in order to obtain U, S and V, build $\hat{R} = U S V^T$

<font color='red'>
$TO DO - Students$

* Choose, implement and evaluate one of the above strategies.
</font>

### Collaborative filtering based on items similarity

In [47]:
""" TRY to predict with item based collaborative filtering """

#We need the corr matrix of movieId's becasue this time it is item based...
corr_movies = train_pivot.corr('pearson')
corr_movies.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1652,1653,1655,1656,1657,1658,1659,1660,1661,1662,1664,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.198057,0.172936,0.128676,0.378934,0.529401,0.153225,0.272667,0.074523,0.204965,0.306748,0.042130,0.007124,0.078223,0.110853,0.262357,0.349287,0.456522,0.116014,0.007776,0.308253,0.285360,0.077972,0.126214,0.113822,0.006976,-0.041602,0.336599,0.290816,-1.596527e-01,0.285213,-0.085398,0.074084,-0.412514,0.359211,0.443578,-0.133631,0.433747,0.290920,0.257710,...,NaN,1.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.188982,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.198057,1.000000,0.172189,0.187792,0.335075,-0.158114,0.140478,0.306391,-0.288128,0.174262,0.285694,0.233521,0.261839,-0.384839,0.184445,0.418330,-0.151709,NaN,-0.342997,0.348263,0.380990,0.090241,-0.160691,0.217958,0.177921,0.325300,0.264580,0.253589,0.203177,4.409586e-01,0.411622,0.352005,0.217418,NaN,-0.156174,0.534522,0.407946,0.343368,0.201778,0.349195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.172936,0.172189,1.000000,-0.134625,0.177084,0.806226,0.017779,-0.182750,-0.089170,0.071563,0.144977,-0.055274,0.363528,0.339946,0.076272,0.531844,0.308377,0.962250,0.289991,0.410974,0.259938,0.402133,-0.294581,0.446424,0.050295,0.241788,0.181684,0.180813,0.309920,-3.033245e-16,0.232591,0.047866,0.554195,-0.866025,NaN,0.500000,-1.000000,0.451677,0.216287,-0.032693,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


####Predict function for item based recommender system:

In [48]:
#PREDICT FUNCTION:
#Convert np
pivot = train_pivot.to_numpy()
corr = corr_movies.to_numpy()

#Enumarate user and movie columns and put in a dictionary:
movie2column = {j:i for i, j in enumerate(train_pivot.columns)}
user2row = {j:i for i, j in enumerate(train_pivot.index)}


def predict_item(pivot, corr, userId, movieId):
  #corr is correlation matrix of movieId's.
    if movieId in movie2column.keys():
        movie = movie2column[movieId]
    else:
        return 2.5
    if userId in user2row.keys():
        user = user2row[userId]
    else:
        return 2.5
    # Normalement le rating est inconnu
    if np.isnan(pivot[user, movie]):
        num = 0
        den = 0
        for m in range(len(corr)):
            if not np.isnan(pivot[user,m]) and not np.isnan(corr[movie,m]):
              #corr of movies
                num += corr[movie,m] * pivot[user,m]
                den += abs(corr[movie,m])
        if den != 0:
            return num/den  
        else:
            return 2.5 # default value
    else:
        # le film a déjà été vu
        print("l'utilisateur", userId, "a déjà vu le film", movieId, "et lui a donné la note de", pivot[user, movie])
        return pivot[user, movie]
        
predict_item(pivot, corr, 1, 1)
predict_item(pivot, corr, 3, 28)

l'utilisateur 1 a déjà vu le film 1 et lui a donné la note de 5.0


0.573635936652926

####Predict:

In [49]:
# Step 3 add the predicted rating to the test set
pred = []
for _, userId, movieId, _ in tqdm(test_ratings[['userId', 'movieId', 'rating']].itertuples()):
    pred += [predict_item(pivot, corr, userId, movieId)]

test_ratings['User based'] = pred
test_ratings.head()

10000it [00:10, 936.55it/s]


,userId,movieId,rating,User based
557,90,900,4,1.897624
6675,90,269,5,3.232988
562,90,289,3,3.032812
62660,90,270,4,1.620365
68756,90,268,4,2.928405


In [50]:
""" Evaluate the result """
# Evaluate the resulte : with classical metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

print("RMSE:",
      np.sqrt(mean_squared_error(test_ratings['rating'], test_ratings['User based'])),
     )

RMSE: 1.7554528828991547


#####HIT RATIO Evaluation:

In [51]:
def predictor(pairs):
    pred = []
    for userId, movieId in zip(pairs[0], pairs[1]):
        pred += [predict_item(pivot, corr, userId, movieId)]
    return np.array(pred)

HR = dict()
hr = HRatio(test_ratings, predictor, 25)
HR['User based'] = hr

  2%|▏         | 15/1000 [00:03<03:35,  4.57it/s]


KeyboardInterrupt: ignored

Hit Ratio is now 0.82

### Matrix factorization

Same approach as in the slides. 

<font color='blue'>
Below is a simple algorithm for factoring a matrix.
</font>

In [52]:
# Matrix factorization from scratch
def matrix_factorization(R, K, steps=10, alpha=0.005):
    '''
    R: rating matrix
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    
    # N: num of User
    N = R.shape[0]
    # M: num of Movie
    M = R.shape[1]

    # P: |U| * K (User features matrix)
    P = np.random.rand(N,K)
    # Q: |D| * K (Item features matrix)
    Q = np.random.rand(M,K).T

    for step in tqdm(range(steps)):
        for i in range(N):
            for j in range(M):
                if not np.isnan(R[i][j]):
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        tmp = P[i][k] + alpha * (2 * eij * Q[k][j])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k])
                        tmp = P[i][k]

    return P, Q.T

In [53]:
# We try first on a toy example
# R: rating matrix
import math
R = [
     [5,3,math.nan,1],
     [4,math.nan,math.nan,1],
     [1,1,math.nan,5],
     [1,math.nan,math.nan,4],
     [0,1,5,4],
     [2,1,3,math.nan],
    ]

R = np.array(R)
# Num of Features
K = 3
 
nP, nQ = matrix_factorization(R, K, steps=10)

nR = np.dot(nP, nQ.T)
nR

100%|██████████| 10/10 [00:00<00:00, 1922.32it/s]


array([[0.65168406, 0.71375366, 0.57213543, 0.69420785],
       [1.10072788, 1.07216813, 0.99339932, 1.19657145],
       [0.71510219, 0.34123903, 0.95818517, 0.80100907],
       [1.07156931, 0.71086163, 0.99830392, 1.23088457],
       [1.55345632, 0.77612013, 1.84753806, 1.77181371],
       [1.21997767, 0.39624047, 1.61195905, 1.41012809]])

Make matrix factorization on pivot

In [54]:
""" TRY to predict with matrix factorization """
K = 10
nP, nQ = matrix_factorization(pivot, K, steps=10)

nR = np.dot(nP, nQ.T)
nR

100%|██████████| 10/10 [01:42<00:00, 10.22s/it]


array([[3.34954207, 3.15366681, 2.57581055, ..., 2.60929353, 2.67794479,
        2.93333953],
       [4.55930075, 3.5095718 , 3.53096796, ..., 3.02370633, 3.3132798 ,
        3.62926453],
       [4.91925695, 3.96889153, 3.31398017, ..., 3.45203226, 4.16050763,
        3.77249212],
       ...,
       [3.37837923, 2.60218273, 2.26156235, ..., 2.31923783, 2.5224817 ,
        2.66740587],
       [3.09812899, 2.40397885, 2.27049751, ..., 2.07489694, 2.11298999,
        2.51821243],
       [3.98742194, 3.3860165 , 2.96970966, ..., 2.87941469, 3.36813741,
        3.09774893]])

In [56]:
def predictor(pairs):
    pred = []
    for userId, movieId in zip(pairs[0], pairs[1]):
        pred += [predict(pivot, nR, userId, movieId)]
    return np.array(pred)

HR = dict()
hr = HRatio(test_ratings, predictor, 25)
HR['User based'] = hr

100%|██████████| 1000/1000 [01:05<00:00, 15.21it/s]


The Hit Ratio @ 25 is 0.86


## Decomposition using latent factor.

We use SVD decomposition

In [58]:
pivot = train_pivot.fillna(0).to_numpy()
max_components = min(train_pivot.shape)-1

In [59]:
from scipy.sparse.linalg import svds

k = 50
assert k<max_components
    
u, s, v_T = svds(pivot, k=k)
nR = u.dot(np.diag(s).dot(v_T))  # output of TruncatedSVD


In [62]:
""" TRY to predict with SVD decomposition """

def predictor(pairs):
    pred = []
    for userId, movieId in zip(pairs[0], pairs[1]):
        pred += [predict(nR,corr, userId, movieId)]
    return np.array(pred)

HR = dict()
hr = HRatio(test_ratings, predictor, 25)
HR['User based'] = hr

  0%|          | 0/1000 [00:00<?, ?it/s]

l'utilisateur 416 a déjà vu le film 1446 et lui a donné la note de 0.2729767772595318
l'utilisateur 416 a déjà vu le film 1553 et lui a donné la note de 0.0906134351996733
l'utilisateur 416 a déjà vu le film 352 et lui a donné la note de 0.6586587906625786
l'utilisateur 416 a déjà vu le film 606 et lui a donné la note de -0.14820800755752764
l'utilisateur 416 a déjà vu le film 1465 et lui a donné la note de -0.03472925879174064
l'utilisateur 416 a déjà vu le film 704 et lui a donné la note de 0.5890835639478127
l'utilisateur 416 a déjà vu le film 1588 et lui a donné la note de 0.00592197419687662
l'utilisateur 416 a déjà vu le film 1601 et lui a donné la note de -0.009112821816713237
l'utilisateur 416 a déjà vu le film 1307 et lui a donné la note de 0.00592197419687662
l'utilisateur 416 a déjà vu le film 361 et lui a donné la note de 0.08153416390711485
l'utilisateur 416 a déjà vu le film 1390 et lui a donné la note de 0.014744505261988321
l'utilisateur 416 a déjà vu le film 1199 et lu

  1%|          | 6/1000 [00:00<01:05, 15.14it/s]

 90 a déjà vu le film 781 et lui a donné la note de -0.09024476329322197
l'utilisateur 90 a déjà vu le film 710 et lui a donné la note de 0.06942569483351078
l'utilisateur 90 a déjà vu le film 257 et lui a donné la note de 0.34947529588567233
l'utilisateur 90 a déjà vu le film 1281 et lui a donné la note de 0.043568728049740206
l'utilisateur 90 a déjà vu le film 1521 et lui a donné la note de 0.03301571608036122
l'utilisateur 90 a déjà vu le film 800 et lui a donné la note de 0.042846064997563466
l'utilisateur 90 a déjà vu le film 1408 et lui a donné la note de -0.0006582457488366326
l'utilisateur 90 a déjà vu le film 189 et lui a donné la note de 0.2068495662348112
l'utilisateur 90 a déjà vu le film 1380 et lui a donné la note de 0.05563841422671095
l'utilisateur 90 a déjà vu le film 1444 et lui a donné la note de 0.0031900107256729497
l'utilisateur 90 a déjà vu le film 29 et lui a donné la note de 0.1412201163882924
l'utilisateur 90 a déjà vu le film 1218 et lui a donné la note de -0

  1%|          | 9/1000 [00:00<00:55, 17.79it/s]

...

472 a déjà vu le film 1375 et lui a donné la note de -0.19466832973733716
l'utilisateur 472 a déjà vu le film 656 et lui a donné la note de -0.059595035103465754
l'utilisateur 472 a déjà vu le film 699 et lui a donné la note de -0.0501330923500186
l'utilisateur 472 a déjà vu le film 464 et lui a donné la note de -0.15695507737798878
l'utilisateur 472 a déjà vu le film 178 et lui a donné la note de -0.624252060167944
l'utilisateur 472 a déjà vu le film 1229 et lui a donné la note de 0.19401959471112648
l'utilisateur 472 a déjà vu le film 357 et lui a donné la note de 0.8145487626461237
l'utilisateur 472 a déjà vu le film 564 et lui a donné la note de 0.5458360255756604
l'utilisateur 472 a déjà vu le film 1021 et lui a donné la note de -0.4853452757488155
l'utilisateur 472 a déjà vu le film 1148 et lui a donné la note de 0.1801611267189466
l'utilisateur 472 a déjà vu le film 1529 et lui a donné la note de -0.09415531236962241
l'utilisateur 472 a déjà vu le film 34 et lui a donné la note 

 98%|█████████▊| 982/1000 [00:29<00:00, 42.47it/s]

a déjà vu le film 1492 et lui a donné la note de 0.03539509517399207
l'utilisateur 234 a déjà vu le film 875 et lui a donné la note de 0.34197759881976414
l'utilisateur 234 a déjà vu le film 961 et lui a donné la note de 0.05275306340578798
l'utilisateur 234 a déjà vu le film 271 et lui a donné la note de 0.4337214369953443
l'utilisateur 234 a déjà vu le film 1390 et lui a donné la note de 0.008621646110313278
l'utilisateur 234 a déjà vu le film 936 et lui a donné la note de 0.23439026037913943
l'utilisateur 234 a déjà vu le film 1515 et lui a donné la note de -0.0060056282053971965
l'utilisateur 234 a déjà vu le film 899 et lui a donné la note de 0.029789039348273566
l'utilisateur 234 a déjà vu le film 452 et lui a donné la note de 0.09835245983819026
l'utilisateur 234 a déjà vu le film 704 et lui a donné la note de 0.04222645386722531
l'utilisateur 234 a déjà vu le film 1229 et lui a donné la note de 0.0060596986779735814
l'utilisateur 234 a déjà vu le film 590 et lui a donné la note

100%|██████████| 1000/1000 [00:29<00:00, 33.47it/s]

a déjà vu le film 1284 et lui a donné la note de 0.22901743362229696
l'utilisateur 234 a déjà vu le film 1498 et lui a donné la note de 0.03539509517399207
l'utilisateur 234 a déjà vu le film 823 et lui a donné la note de 0.07338858872880458
l'utilisateur 234 a déjà vu le film 1618 et lui a donné la note de -0.0024935542758078772
l'utilisateur 234 a déjà vu le film 192 et lui a donné la note de -0.22202812860384769
l'utilisateur 655 a déjà vu le film 232 et lui a donné la note de -0.002035362651198478
l'utilisateur 655 a déjà vu le film 1332 et lui a donné la note de 0.02285686314105972
l'utilisateur 655 a déjà vu le film 866 et lui a donné la note de 0.025998337644113172
l'utilisateur 655 a déjà vu le film 1439 et lui a donné la note de 0.4706125368929126
l'utilisateur 655 a déjà vu le film 1352 et lui a donné la note de 0.01990687013552126
l'utilisateur 655 a déjà vu le film 1413 et lui a donné la note de 0.02496095621778727
l'utilisateur 655 a déjà vu le film 892 et lui a donné la n